In [1]:
products = [ 
          # (product_id, product_name, brand_id)  
         (1, 'iPhone', 100),
         (2, 'Galaxy', 200),
         (3, 'Redme', 300), # orphan record, no matching brand
         (4, 'Pixel', 400),
]

# spark session. For any spark app, there should be ONLY ONE Spark Contect [sc], there can be many spark session [spark]
# DAta Frame - structured data
# strcutred data = [Schema ] + data rows 
productDf = spark.createDataFrame(data=products, schema=["product_id", "product_name", "brand_id"])

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1635236588415_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
productDf.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- product_id: long (nullable = true)
 |-- product_name: string (nullable = true)
 |-- brand_id: long (nullable = true)

In [3]:
productDf.show() # display first 20 results

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------+--------+
|product_id|product_name|brand_id|
+----------+------------+--------+
|         1|      iPhone|     100|
|         2|      Galaxy|     200|
|         3|       Redme|     300|
|         4|       Pixel|     400|
+----------+------------+--------+

In [4]:
# behind any dataframe, there is always RDD, Rows
productDf.rdd.collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(product_id=1, product_name='iPhone', brand_id=100), Row(product_id=2, product_name='Galaxy', brand_id=200), Row(product_id=3, product_name='Redme', brand_id=300), Row(product_id=4, product_name='Pixel', brand_id=400)]

In [5]:
# when we query/apply any transformation on DataFrame, internally the logics are applied to RDD

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:

print(productDf.rdd.getNumPartitions())
print(productDf.rdd.glom().collect())
productDf.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2
[[Row(product_id=1, product_name='iPhone', brand_id=100), Row(product_id=2, product_name='Galaxy', brand_id=200)], [Row(product_id=3, product_name='Redme', brand_id=300), Row(product_id=4, product_name='Pixel', brand_id=400)]]
+----------+------------+--------+
|product_id|product_name|brand_id|
+----------+------------+--------+
|         1|      iPhone|     100|
|         2|      Galaxy|     200|
|         3|       Redme|     300|
|         4|       Pixel|     400|
+----------+------------+--------+

In [10]:
# select query, product_name, brand_id
# select statement, returns a new dataframe, without modifying the productDf
# producDf, in general, DAtaFrame, is immutable - cannot be changed, READ only, constant
# in SPARK, we cannot change the original data once it is loaded into memory
# return new dataframe, that has its own rdd, has its own partition, its own SCHEMA, rows
productNameDf = productDf.select('product_name', 'brand_id')
productNameDf.printSchema()
print(productNameDf.rdd.getNumPartitions())
print(productNameDf.rdd.glom().collect())
productNameDf.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- product_name: string (nullable = true)
 |-- brand_id: long (nullable = true)

2
[[Row(product_name='iPhone', brand_id=100), Row(product_name='Galaxy', brand_id=200)], [Row(product_name='Redme', brand_id=300), Row(product_name='Pixel', brand_id=400)]]
+------------+--------+
|product_name|brand_id|
+------------+--------+
|      iPhone|     100|
|      Galaxy|     200|
|       Redme|     300|
|       Pixel|     400|
+------------+--------+

In [12]:
# where, filter [both are same, alias]
# return new data frame
# python syntax in where/filter
filterDf = productDf.where (productDf.brand_id >= 300)
filterDf.show()



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------+--------+
|product_id|product_name|brand_id|
+----------+------------+--------+
|         3|       Redme|     300|
|         4|       Pixel|     400|
+----------+------------+--------+

In [14]:
# where with SQL Statement 
filterDf = productDf.where ("brand_id  >= 300")
filterDf.show()

# performance wise SQL and python syntax all same

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------+--------+
|product_id|product_name|brand_id|
+----------+------------+--------+
|         3|       Redme|     300|
|         4|       Pixel|     400|
+----------+------------+--------+

In [15]:

brands = [
    #(brand_id, brand_name)
    (100, "Apple"),
    (200, "Samsung"),
    (400, "Google"),
    (500, "Sony"), # no matching products
]

brandDf = spark.createDataFrame(data=brands, schema=["brand_id", "brand_name"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
productDf.show()
brandDf.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------+--------+
|product_id|product_name|brand_id|
+----------+------------+--------+
|         1|      iPhone|     100|
|         2|      Galaxy|     200|
|         3|       Redme|     300|
|         4|       Pixel|     400|
+----------+------------+--------+

+--------+----------+
|brand_id|brand_name|
+--------+----------+
|     100|     Apple|
|     200|   Samsung|
|     400|    Google|
|     500|      Sony|
+--------+----------+

In [17]:
# Inner Join
# productDf is left
# brandDf is right
# select/pick only matching record, discord if no matches found
resultDf = productDf.join(brandDf, productDf["brand_id"] ==  brandDf["brand_id"], "inner")
resultDf.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------+--------+--------+----------+
|product_id|product_name|brand_id|brand_id|brand_name|
+----------+------------+--------+--------+----------+
|         1|      iPhone|     100|     100|     Apple|
|         2|      Galaxy|     200|     200|   Samsung|
|         4|       Pixel|     400|     400|    Google|
+----------+------------+--------+--------+----------+

In [18]:
# Outer Join, Full Outer Outer, [Left outer + Right outer]
# pick all records from left dataframe, and also right dataframe
# if no matches found, it fills null data for not matched records
productDf.join(brandDf, productDf["brand_id"] ==  brandDf["brand_id"], "outer").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------+--------+--------+----------+
|product_id|product_name|brand_id|brand_id|brand_name|
+----------+------------+--------+--------+----------+
|      null|        null|    null|     500|      Sony|
|         1|      iPhone|     100|     100|     Apple|
|         2|      Galaxy|     200|     200|   Samsung|
|         4|       Pixel|     400|     400|    Google|
|         3|       Redme|     300|    null|      null|
+----------+------------+--------+--------+----------+

In [19]:
# Left, Left Outer join 
# picks all records from left, if no matches found, it fills null for right data
productDf.join(brandDf, productDf["brand_id"] ==  brandDf["brand_id"], "leftouter").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------+--------+--------+----------+
|product_id|product_name|brand_id|brand_id|brand_name|
+----------+------------+--------+--------+----------+
|         1|      iPhone|     100|     100|     Apple|
|         2|      Galaxy|     200|     200|   Samsung|
|         4|       Pixel|     400|     400|    Google|
|         3|       Redme|     300|    null|      null|
+----------+------------+--------+--------+----------+

In [20]:
# Right, Right outer Join
# picks all the records from right, if no matches found, fills left data with null
productDf.join(brandDf, productDf["brand_id"] ==  brandDf["brand_id"], "rightouter").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------+--------+--------+----------+
|product_id|product_name|brand_id|brand_id|brand_name|
+----------+------------+--------+--------+----------+
|      null|        null|    null|     500|      Sony|
|         1|      iPhone|     100|     100|     Apple|
|         2|      Galaxy|     200|     200|   Samsung|
|         4|       Pixel|     400|     400|    Google|
+----------+------------+--------+--------+----------+

In [21]:
# left semi join
# join in general convention, it pull the records from both right and left, join them based on condition
# left semi join, join left and right based on condition, however it pull the records only from left side

# it is similar to innerjoin, but pick/project records only from left
# we can't see brand_id, brand_name from brands df
productDf.join(brandDf, productDf["brand_id"] ==  brandDf["brand_id"], "leftsemi").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------+--------+
|product_id|product_name|brand_id|
+----------+------------+--------+
|         1|      iPhone|     100|
|         2|      Galaxy|     200|
|         4|       Pixel|     400|
+----------+------------+--------+

In [22]:
# left anti join: exact opposite to semi join
# picks the records that doesn't have match on the right side
productDf.join(brandDf, productDf["brand_id"] ==  brandDf["brand_id"], "leftanti").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------+--------+
|product_id|product_name|brand_id|
+----------+------------+--------+
|         3|       Redme|     300|
+----------+------------+--------+